In [16]:
import os
import json
import shutil
import copy
from zipfile import ZipFile
from modules.utils import *
from modules.geometry import *

In [17]:
path = "/Users/samo/Documents/School/Bioinf/CVAT_polygon_interpolator/data/task_sonoscape_2021-09_2021-10-18_003_20211001_071332_27.avi_backup_2022_04_08_11_05_32 (1).zip"

In [18]:
# comment out unwanted
shapes_lst = [
    "lungslidingpresent",
    "lungslidingabsent",
    "aline",
    "bline"
]

In [19]:
with ZipFile(path) as myzip:
    with myzip.open("annotations.json") as json_file:
        annots = json.load(json_file)


In [20]:
shapes = copy.deepcopy(annots[0]["shapes"])
filtered_shapes = [
    shape
    for shape in shapes
    if shape["type"] == "polygon"
    and shape["label"] in shapes_lst
]
indicies = get_indicies(shapes)

In [21]:
shapes_by_frame = group_shapes_by_frame(filtered_shapes)
remove_duplicates(shapes_by_frame)
assign_id(shapes_by_frame)
interpolator(shapes_by_frame, indicies, annots)

In [22]:
# create output folder if it doesn't exist
if not os.path.exists("output"):
    os.makedirs("output")

# copy the original file to output folder
shutil.copy(path, "output/" + path.split("/")[-1])

remove_from_zip("output/" + path.split("/")[-1], "annotations.json")
with zipfile.ZipFile("output/" + path.split("/")[-1], 'a') as newzip:
    newzip.writestr("annotations.json", json.dumps(annots))
